# training_dir_staging

### Tag Counter
----

In [2]:
import os
from collections import Counter

def extract_tags_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        tags = content.split(', ')
        return tags

def process_directory(directory_path):
    all_tags = []
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                tags = extract_tags_from_file(file_path)
                all_tags.extend(tags)
    return all_tags

def list_tags_frequency(tags):
    tag_counter = Counter(tags)
    sorted_tags = sorted(tag_counter.items(), key=lambda x: x[1], reverse=True)
    return sorted_tags

# Specify the directory path
directory_path = r'C:\Users\kade\Desktop\training_dir_staging'

# Process the directory and extract tags
all_tags = process_directory(directory_path)

# List tags and their frequencies in descending order
sorted_tags = list_tags_frequency(all_tags)

# Print the results
for tag, frequency in sorted_tags:
    print(f'{tag}: {frequency} times')

dagasi: 200 times
bodily fluids: 197 times
rating_explicit: 191 times
hi res: 185 times
genitals: 177 times
blush: 171 times
mammal: 166 times
genital fluids: 165 times
male: 163 times
female: 155 times
sex: 135 times
anthro: 131 times
penis: 128 times
penetration: 123 times
male female: 120 times
fur: 120 times
pussy: 119 times
nintendo: 118 times
vaginal fluids: 115 times
pokemon: 115 times
pokemon (species): 115 times
sweat: 114 times
penile: 111 times
male penetrating: 108 times
duo: 102 times
tears: 101 times
cum: 97 times
anus: 93 times
cum inside: 93 times
female penetrated: 93 times
male penetrating female: 92 times
penile penetration: 92 times
vaginal: 91 times
vaginal penetration: 87 times
interspecies: 85 times
tail: 84 times
butt: 82 times
tongue: 82 times
group: 81 times
solo focus: 80 times
heart symbol: 78 times
digital media (artwork): 78 times
human: 77 times
balls: 76 times
looking pleasured: 76 times
open mouth: 74 times
young: 73 times
spreading: 70 times
saliva: 70

### Replace tags and remove duplicates
----

In [35]:
import os
import re

def process_files(directory, old_tag, new_tag):
    try:
        for entry in os.listdir(directory):
            entry_path = os.path.join(directory, entry)

            if os.path.isdir(entry_path):
                process_files(entry_path, old_tag, new_tag)

            elif os.path.isfile(entry_path) and entry.endswith(".txt"):
                with open(entry_path, 'r', encoding='utf-8') as f:
                    content = f.read()

                if old_tag:
                    content = re.sub(r'\b' + re.escape(old_tag) + r'\b', new_tag, content)

                tag_pattern = re.compile(r'(\b\w+\b)(?:(?:,|\s)+\1)+')
                content = re.sub(tag_pattern, r'\1', content)

                with open(entry_path, 'w', encoding='utf-8') as f:
                    f.write(content)

    except Exception as e:
        print(f"Error processing directory {directory}: {e}\n")

# Directory path
directory_path = r'C:\Users\kade\Desktop\training_dir_staging'

process_files(directory_path, 'were', 'sparkledog')
process_files(directory_path, 'werecanine', 'sparkledog')
process_files(directory_path, 'werewolf', 'sparkledog')

### Insert tag
----

In [1]:
import os

# Function to insert a specified tag in text files in subdirectories
def insert_tag_in_files(directory, tag_to_insert):
    try:
        for entry in os.listdir(directory):
            entry_path = os.path.join(directory, entry)

            if os.path.isdir(entry_path):
                insert_tag_in_files(entry_path, tag_to_insert)

            elif os.path.isfile(entry_path) and entry.endswith(".txt"):

                with open(entry_path, 'r', encoding='utf-8') as f:
                    content = f.read()

                # Insert the specified tag
                content = tag_to_insert + ', ' + content

                # Write back to the file
                with open(entry_path, 'w', encoding='utf-8') as f:
                    f.write(content)

    except Exception as e:
        print(f"Error processing directory {directory}: {e}\n")

directory_path = r'C:\Users\kade\Desktop\training_dir_staging'

# Execute the function with the desired tag
insert_tag_in_files(directory_path, 'hamgas')

### Escape parentheses
----

Recursively escape unescaped parentheses in all '.txt' files within the specified directory and its subdirectories. 

In [3]:
import os
import re

def escape_parentheses(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Escape unescaped parentheses
    content = re.sub(r'(?<!\\)([()])', r'\\\1', content)

    with open(file_path, 'w') as file:
        file.write(content)

def process_directory(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                escape_parentheses(file_path)

                # Recurse into subdirectories
                process_directory(os.path.join(root, file))

directory_path = r'C:\Users\kade\Desktop\training_dir_staging'
process_directory(directory_path)

### Replace underscores with spaces
----

Recursively replaces underscores with spaces in the content of text files in the specified directory and its subdirectories,
excluding specified tags.

In [4]:
import os

excluded_tags = [
    "rating_safe",
    "rating_explicit",
    "rating_questionable"
]

def replace_underscores_with_spaces(directory_path):
    for root, dirs, files in os.walk(directory_path):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)

                # Read the content of the file
                with open(file_path, 'r') as file:
                    content = file.read()

                # Replace underscores with spaces, excluding specific tags
                for tag in excluded_tags:
                    replacement = tag.replace('_', ' ')
                    content = content.replace(replacement, tag)

                # Write the modified content back to the file
                with open(file_path, 'w') as file:
                    file.write(content)

# Specify the directory path
directory_path = r'C:\Users\kade\Desktop\training_dir_staging'

# Call the function to recursively replace underscores with spaces (excluding specified tags)
replace_underscores_with_spaces(directory_path)

### Fix tags that need underscores after that! 🐱
----

In [12]:
import os
import fileinput

# Function to recursively replace text in *.txt files
def replace_text_in_files(directory):
    for subdir, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(subdir, file)
                with fileinput.FileInput(file_path, inplace=True) as file:
                    for line in file:
                        print(line.replace("rating safe", "rating_safe")
                                   .replace("rating questionable", "rating_questionable")
                                   .replace("rating explicit", "rating_explicit"), end='')

# Replace text in the specified directory
replace_text_in_files(r'C:\Users\kade\Desktop\training_dir_staging')

### Remove extra file extension before .txt
----

In [5]:
import os

def rename_files(directory_path):
    """
    Recursively renames files with additional image extensions in the specified directory and its subdirectories.
    """
    for root, dirs, files in os.walk(directory_path):
        for filename in files:
            if filename.endswith('.txt'):
                # Extract the base name without extension
                base_name, extension = os.path.splitext(filename)

                # Check if the file has an additional image extension
                if base_name.endswith(('.png', '.jpg', '.jpeg', '.webp', '.gif')):
                    # Construct the new filename with only the txt extension
                    new_filename = base_name[:-4] + '.txt'

                    # Construct the full file paths
                    old_path = os.path.join(root, filename)
                    new_path = os.path.join(root, new_filename)

                    # Rename the file
                    os.rename(old_path, new_path)

# Specify the directory path
directory_path = r'C:\Users\kade\Desktop\training_dir_staging'

# Call the function to recursively rename files
rename_files(directory_path)

### Newlines to commas
----

Recursively modify the content of '.txt' files in the specified directory and its subdirectories by replacing newlines with commas and spaces. 

In [14]:
import os

def process_directory(directory):
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                
                # Read the content of the file
                with open(file_path, 'r') as file:
                    content = file.read()
                
                # Replace newline with a comma and space
                modified_content = content.replace('\n', ', ')
                
                # Write the modified content back to the file
                with open(file_path, 'w') as file:
                    file.write(modified_content)

# Directory path
directory_path = r'C:\Users\kade\Desktop\training_dir_staging'

# Recursively process the directory and its subdirectories
process_directory(directory_path)

### Remove tags ⚠️
----

In [18]:
import os

def remove_tags(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    tags_to_remove = [
        "partially",
        "pokemon \(species\)",
        "generation",
        "pokephilia",
        "pokemon",
        "nintendo",
        "eeveelution",
        "uncensored",
        "translated",
        "partially translated",
        "translation request",
        "16 10",
        "16 9",
        "9 16",
        "6 5",
        "5 6",
        "4 3",
        "4 5",
        "3 4",
        "2 3",
        "2 1",
        "1 1",
        "absurd res",
        "hi res",
        "elden ring",
        "fromsoftware",
        "canid",
        "canis",
        "mammal",
        "unwanted erection",
        "lighting",
        "shaded",
        "widescreen"
    ]

    for gen in range(1, 9):
        tags_to_remove.append(f"generation {gen} pokemon")

    for year in range(1996, 2025):
        tags_to_remove.append(str(year))

    removal_actions = []

    for tag in tags_to_remove:
        if tag in content:
            content = content.replace(tag, '')

            tag_comma = f"{tag}, "
            content = content.replace(tag_comma, '')

            removal_actions.append(f'Removed tag "{tag}" and ", " after it from file: {file_path}')

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

    return removal_actions

def process_directory(directory):
    all_removal_actions = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                removal_actions = remove_tags(file_path)
                all_removal_actions.extend(removal_actions)

    for action in all_removal_actions:
        print(action)

# Provide the path to the directory
directory_path = r'C:\Users\kade\Desktop\training_dir_staging'

# Recursively remove tags and ", " after tags from *.txt files in the specified directory and print removal actions
process_directory(directory_path)


Removed tag "partially" and ", " after it from file: C:\Users\kade\Desktop\training_dir_staging\1_dagasi\0348b27ad1632fe662b1ea80d4f99f62.txt
Removed tag "partially" and ", " after it from file: C:\Users\kade\Desktop\training_dir_staging\1_dagasi\03f5be6578929a52c5f596e0cc7bb3de.txt
Removed tag "partially" and ", " after it from file: C:\Users\kade\Desktop\training_dir_staging\1_dagasi\0bab1f7aa0d6a1f786621ef4e067acf2.txt
Removed tag "partially" and ", " after it from file: C:\Users\kade\Desktop\training_dir_staging\1_dagasi\0edc7a89c4b9e5f8d1c3b58f9803f9a3.txt
Removed tag "partially" and ", " after it from file: C:\Users\kade\Desktop\training_dir_staging\1_dagasi\746d2a114cc33f5c234e60dfcc55fd1f.txt
Removed tag "partially" and ", " after it from file: C:\Users\kade\Desktop\training_dir_staging\1_dagasi\7943e0187c509d4d1fec512ec4e6b98c.txt
Removed tag "partially" and ", " after it from file: C:\Users\kade\Desktop\training_dir_staging\1_dagasi\8dee9f2614ca6f893a099904ce3991cb.txt
Remove

### Replace `, ,` with `,` after that mess. 😼
----

In [24]:
import os

# Start directory
start_dir = r'C:\Users\kade\Desktop\training_dir_staging'

# Function to replace text in *.txt files
def replace_text_in_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                # Replace ', ,' with ','
                content = content.replace(', ,', ',')
                content = content.replace(',  ,', ',')
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(content)

# Run the function
replace_text_in_files(start_dir)
